# **0️⃣. Installation** des bibliothèques

In [ ]:
0. # Installer nltk (si nécessaire)
!pip install nltk pandas

1 # Importer les bibliothèques
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

 # Télécharger les ressources nltk (une seule fois)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
from google.colab import files

2# 📂 Demande à l'utilisateur de téléverser son fichier
uploaded = files.upload()

Saving livres_final_update_final avec auteur.xlsx to livres_final_update_final avec auteur (1).xlsx


# 1️⃣ **Vérifier le fichier source (df.head())**
🎯 **Objectif** : Regarder la structure du fichier et confirmer les colonnes existantes.

In [ ]:
import pandas as pd
from tqdm import tqdm

# Charger le fichier (remplace "mon_fichier.csv" par le vrai nom)
df = pd.read_excel("livres_final_update_final avec auteur (1).xlsx")

# Vérifier les premières lignes
print("🔍 Aperçu des données :")
print(df.head())

# Vérifier les colonnes disponibles
print("\n📌 Colonnes disponibles :")
print(df.columns)



🔍 Aperçu des données :
                                           Titre             Auteur  \
0           Taupe & Mulot. Atteindre les sommets      Henri Meunier   
1                   La guerre et la paix. Vol. 1       Léon Tolstoï   
2                              L'âge de la forêt  Charline Collette   
3  Des pas dans mon ciel bleu : CE1, série jaune  Nadine Brun-Cosme   
4             10 bonnes raisons de te détester !         Emma Green   

               Éditeur                                        Description  \
0               Hélium  - Aaaaahhhh... ! - Mulot ! Mulot ! Ça va, peti...   
1            Gallimard  - Ah ! enlevez ces... enlevez donc ces... (Ell...   
2         Joie de lire  - Alors le grand chêne est plus vieux que toi ...   
3               Hatier  - Alors voilà, a conclu Papa en m'écartant trè...   
4  Editions Addictives  - Arrête de te croire irrésistible, art. - Arr...   

   Prix                                          Image URL  \
0  13.9  https://media.el

# 2️⃣**Ajouter une colonne "Publication"**
 🎯 **Objectif** :
 Comme nous n'avons pas encore les années de publication, nous allons ajouter une colonne vide qui sera remplie plus tard avec l'API Google Books.

In [ ]:
# Ajouter une colonne vide pour l'année de publication
df["Publication"] = None

# Vérifier que la colonne a bien été ajoutée
print("✅ Colonne 'Publication' ajoutée avec succès !")
print(df.head())



✅ Colonne 'Publication' ajoutée avec succès !
                                           Titre             Auteur  \
0           Taupe & Mulot. Atteindre les sommets      Henri Meunier   
1                   La guerre et la paix. Vol. 1       Léon Tolstoï   
2                              L'âge de la forêt  Charline Collette   
3  Des pas dans mon ciel bleu : CE1, série jaune  Nadine Brun-Cosme   
4             10 bonnes raisons de te détester !         Emma Green   

               Éditeur                                        Description  \
0               Hélium  - Aaaaahhhh... ! - Mulot ! Mulot ! Ça va, peti...   
1            Gallimard  - Ah ! enlevez ces... enlevez donc ces... (Ell...   
2         Joie de lire  - Alors le grand chêne est plus vieux que toi ...   
3               Hatier  - Alors voilà, a conclu Papa en m'écartant trè...   
4  Editions Addictives  - Arrête de te croire irrésistible, art. - Arr...   

   Prix                                          Image URL  \
0 

# 3️⃣ **Utiliser l'API Google Books pour récupérer les auteurs inconnus**
🎯 Objectif : Rechercher les livres avec "Auteur inconnu" dans la colonne "Auteur" et utiliser Google Books API pour trouver les vrais auteurs.

In [ ]:
import requests
import time

# Clé API Google Books (remplace "YOUR_API_KEY" par ta clé)
API_KEY = "YOUR_API_KEY"

# URL de l'API Google Books
GOOGLE_BOOKS_API_URL = "https://www.googleapis.com/books/v1/volumes"

def get_book_info(title):
    """Recherche un livre sur Google Books et retourne l'auteur trouvé."""
    params = {
        "q": f"{title} livre",
        "maxResults": 1,
        "key": API_KEY
    }
    response = requests.get(GOOGLE_BOOKS_API_URL, params=params)
    data = response.json()

    if "items" in data:
        book_info = data["items"][0]["volumeInfo"]
        auteur = ", ".join(book_info.get("authors", ["Auteur inconnu"]))
        return auteur
    return "Auteur inconnu"

    # Début du chronomètre
start_time = time.time()

# Sélectionner uniquement les lignes où l'auteur est "Auteur inconnu"
livres_sans_auteur = df[df["Auteur"].str.lower() == "auteur inconnu"]

# Barre de progression
for index, row in tqdm(livres_sans_auteur.iterrows(), total=len(livres_sans_auteur), desc="🔄 Recherche des auteurs via API"):
    df.at[index, "Auteur"] = get_book_info(row["Titre"])

# Calcul du temps d'exécution
elapsed_time = time.time() - start_time

# Vérifier les résultats après mise à jour
auteurs_trouves = df[df["Auteur"].str.lower() != "auteur inconnu"]

print("\n✅ Mise à jour des auteurs terminée.")
print(f"⏳ Temps total : {elapsed_time:.2f} secondes")
print(f"📌 Nombre d'auteurs trouvés : {len(auteurs_trouves)}")
print(f"📌 Nombre restant 'Auteur inconnu' : {len(df[df['Auteur'].str.lower() == 'auteur inconnu'])}")

🔄 Recherche des auteurs via API: 100%|██████████| 7983/7983 [02:31<00:00, 52.82it/s]



✅ Mise à jour des auteurs terminée.
⏳ Temps total : 151.22 secondes
📌 Nombre d'auteurs trouvés : 63230
📌 Nombre restant 'Auteur inconnu' : 7983


# 4️⃣ **Utiliser l'API Google Books pour récupérer les descriptions**
🎯 **Objectif** : Remplacer les descriptions incorrectes ("pas de description disponible", "expédié par", etc.) par des résumés corrects à l’aide de Google Books API.

In [ ]:
# Renommer la colonne "Description" en "Résumé"
df.rename(columns={"Description": "Résumé"}, inplace=True)

# Vérifier le changement
print("✅ La colonne 'Description' a été renommée en 'Résumé'")
print(df.head())


✅ La colonne 'Description' a été renommée en 'Résumé'
                                           Titre             Auteur  \
0           Taupe & Mulot. Atteindre les sommets      Henri Meunier   
1                   La guerre et la paix. Vol. 1       Léon Tolstoï   
2                              L'âge de la forêt  Charline Collette   
3  Des pas dans mon ciel bleu : CE1, série jaune  Nadine Brun-Cosme   
4             10 bonnes raisons de te détester !         Emma Green   

               Éditeur                                             Résumé  \
0               Hélium  - Aaaaahhhh... ! - Mulot ! Mulot ! Ça va, peti...   
1            Gallimard  - Ah ! enlevez ces... enlevez donc ces... (Ell...   
2         Joie de lire  - Alors le grand chêne est plus vieux que toi ...   
3               Hatier  - Alors voilà, a conclu Papa en m'écartant trè...   
4  Editions Addictives  - Arrête de te croire irrésistible, art. - Arr...   

   Prix                                          Image U

In [ ]:
def get_book_summary(title, isbn=None):
    """Recherche un livre sur Google Books et retourne l'auteur et le résumé."""

    if isbn:
        params = {"q": f"isbn:{isbn}", "maxResults": 1, "key": API_KEY}
    else:
        params = {"q": f"{title} livre résumé", "maxResults": 1, "key": API_KEY}

    response = requests.get(GOOGLE_BOOKS_API_URL, params=params)
    data = response.json()

    if "items" in data:
        book_info = data["items"][0]["volumeInfo"]
        auteur = ", ".join(book_info.get("authors", ["Auteur inconnu"]))
        resume = book_info.get("description", "Résumé non disponible")
        return auteur, resume

    return "Auteur inconnu", "Résumé non disponible"
# Début du chronomètre
start_time = time.time()

# Liste des valeurs à remplacer dans la colonne "Résumé"
descriptions_vides = ["description non disponible", "expédié par", "pas de description disponible"]

# Sélectionner uniquement les lignes où le résumé est incorrect
livres_sans_resume = df[df["Résumé"].str.lower().isin(descriptions_vides)]

# Barre de progression
for index, row in tqdm(livres_sans_resume.iterrows(), total=len(livres_sans_resume), desc="📖 Recherche des résumés via API"):
    isbn = row["ISBN"] if "ISBN" in df.columns and pd.notna(row["ISBN"]) else None
    _, resume = get_book_summary(row["Titre"], isbn)
    df.at[index, "Résumé"] = resume

# Calcul du temps d'exécution
elapsed_time = time.time() - start_time

# Vérifier les résultats après mise à jour
resumes_restants = df[df["Résumé"].str.lower().isin(descriptions_vides)]

print("\n✅ Mise à jour des résumés terminée.")
print(f"⏳ Temps total : {elapsed_time:.2f} secondes")
print(f"📌 Nombre restant 'Résumé non disponible' après mise à jour : {len(resumes_restants)}")


📖 Recherche des résumés via API: 100%|██████████| 1771/1771 [00:30<00:00, 58.52it/s]



✅ Mise à jour des résumés terminée.
⏳ Temps total : 30.85 secondes
📌 Nombre restant 'Résumé non disponible' après mise à jour : 0


# 5️⃣**Utiliser l'API Google Books pour récupérer les dates de publication**
🎯 **Objectif** : Remplacer les valeurs None dans la colonne "Publication" par la vraie année de publication grâce à l’API Google Books.

In [ ]:
def get_book_details(title, isbn=None):
    """Recherche un livre sur Google Books et retourne l'auteur, le résumé et la date de publication."""

    if isbn:
        params = {"q": f"isbn:{isbn}", "maxResults": 1, "key": API_KEY}
    else:
        params = {"q": f"{title} livre", "maxResults": 1, "key": API_KEY}

    response = requests.get(GOOGLE_BOOKS_API_URL, params=params)
    data = response.json()

    if "items" in data:
        book_info = data["items"][0]["volumeInfo"]
        auteur = ", ".join(book_info.get("authors", ["Auteur inconnu"]))
        resume = book_info.get("description", "Résumé non disponible")
        date_publication = book_info.get("publishedDate", "Date inconnue")
        return auteur, resume, date_publication

    return "Auteur inconnu", "Résumé non disponible", "Date inconnue"

# Début du chronomètre
start_time = time.time()

# Sélectionner uniquement les lignes où la date de publication est "None" ou "Date inconnue"
livres_sans_date = df[df["Publication"].isna() | (df["Publication"] == "Date inconnue")]

# Barre de progression
for index, row in tqdm(livres_sans_date.iterrows(), total=len(livres_sans_date), desc="📅 Recherche des dates de publication via API"):
    isbn = row["ISBN"] if "ISBN" in df.columns and pd.notna(row["ISBN"]) else None
    _, _, date_publication = get_book_details(row["Titre"], isbn)
    df.at[index, "Publication"] = date_publication

# Calcul du temps d'exécution
elapsed_time = time.time() - start_time

# Vérifier les résultats après mise à jour
dates_restantes = df[df["Publication"].isna() | (df["Publication"] == "Date inconnue")]

print("\n✅ Mise à jour des dates terminée.")
print(f"⏳ Temps total : {elapsed_time:.2f} secondes")
print(f"📌 Nombre restant 'Date inconnue' après mise à jour : {len(dates_restantes)}")



📅 Recherche des dates de publication via API: 100%|██████████| 71213/71213 [22:04<00:00, 53.77it/s]



✅ Mise à jour des dates terminée.
⏳ Temps total : 1324.48 secondes
📌 Nombre restant 'Date inconnue' après mise à jour : 71213


In [ ]:
# Vérifier les premiers livres avec "Date inconnue"
print("\n📌 Exemples de livres sans date trouvée :")
print(df[df["Publication"] == "Date inconnue"].head(10))


📌 Exemples de livres sans date trouvée :
                                            Titre             Auteur  \
0            Taupe & Mulot. Atteindre les sommets      Henri Meunier   
1                    La guerre et la paix. Vol. 1       Léon Tolstoï   
2                               L'âge de la forêt  Charline Collette   
3   Des pas dans mon ciel bleu : CE1, série jaune  Nadine Brun-Cosme   
4              10 bonnes raisons de te détester !         Emma Green   
5                              La flûte enchantée       Pierre Coran   
6                                     Le bonhomme  Stephen Vuillemin   
7                                       Oui & non      Elisha Cooper   
8               Comment on construit une maison ?        César Canet   
9  Vie et mort de l'homme qui tua John F. Kennedy  Anne-James Chaton   

                  Éditeur                                             Résumé  \
0                  Hélium  - Aaaaahhhh... ! - Mulot ! Mulot ! Ça va, peti...   
1    

In [ ]:
import re

def get_book_year(title, isbn=None):
    """Recherche un livre sur Google Books et retourne l'année de publication."""

    if isbn:
        params = {"q": f"isbn:{isbn}", "maxResults": 1, "key": API_KEY}
    else:
        params = {"q": f"{title} livre", "maxResults": 1, "key": API_KEY}

    response = requests.get(GOOGLE_BOOKS_API_URL, params=params)
    data = response.json()

    if "items" in data:
        book_info = data["items"][0]["volumeInfo"]
        date_publication = book_info.get("publishedDate", "Date inconnue")

        # Extraire uniquement l'année si un format plus long est trouvé
        match = re.search(r"\d{4}", date_publication)  # Recherche une année (4 chiffres)
        if match:
            return match.group(0)  # Retourne uniquement l'année

    return "Date inconnue"

# Début du chronomètre
start_time = time.time()

# Sélectionner uniquement les lignes où la date est "Date inconnue" ou "non"
livres_sans_date = df[df["Publication"].isna() | (df["Publication"].str.lower() == "non") | (df["Publication"] == "Date inconnue")]

# Barre de progression
for index, row in tqdm(livres_sans_date.iterrows(), total=len(livres_sans_date), desc="📅 Recherche des années de publication via API"):
    isbn = row["ISBN"] if "ISBN" in df.columns and pd.notna(row["ISBN"]) else None
    year = get_book_year(row["Titre"], isbn)
    df.at[index, "Publication"] = year

# Calcul du temps d'exécution
elapsed_time = time.time() - start_time

# Vérifier les résultats après mise à jour
dates_restantes = df[df["Publication"] == "Date inconnue"]

print("\n✅ Mise à jour des années terminée.")
print(f"⏳ Temps total : {elapsed_time:.2f} secondes")
print(f"📌 Nombre restant 'Date inconnue' après mise à jour : {len(dates_restantes)}")


📅 Recherche des années de publication via API: 100%|██████████| 71213/71213 [22:13<00:00, 53.40it/s]



✅ Mise à jour des années terminée.
⏳ Temps total : 1333.57 secondes
📌 Nombre restant 'Date inconnue' après mise à jour : 71213


In [ ]:
# Supprimer la colonne "Publication"
df.drop(columns=["Publication"], inplace=True)

# Vérifier que la colonne a bien été supprimée
print("✅ Colonne 'Publication' supprimée avec succès !")
print(df.head())


✅ Colonne 'Publication' supprimée avec succès !
                                           Titre             Auteur  \
0           Taupe & Mulot. Atteindre les sommets      Henri Meunier   
1                   La guerre et la paix. Vol. 1       Léon Tolstoï   
2                              L'âge de la forêt  Charline Collette   
3  Des pas dans mon ciel bleu : CE1, série jaune  Nadine Brun-Cosme   
4             10 bonnes raisons de te détester !         Emma Green   

               Éditeur                                             Résumé  \
0               Hélium  - Aaaaahhhh... ! - Mulot ! Mulot ! Ça va, peti...   
1            Gallimard  - Ah ! enlevez ces... enlevez donc ces... (Ell...   
2         Joie de lire  - Alors le grand chêne est plus vieux que toi ...   
3               Hatier  - Alors voilà, a conclu Papa en m'écartant trè...   
4  Editions Addictives  - Arrête de te croire irrésistible, art. - Arr...   

   Prix                                          Image URL  \


# **📌 Plan des étapes pour le NLP Processing**
1️⃣ **Normaliser le texte** : suppression des accents, mise en minuscules

2️⃣ **Tokenisation** : découper les textes en mots

3️⃣ **Suppression des stopwords** : mots non significatifs comme "le", "de", "et"

4️⃣ **Lemmatisation** : réduction des mots à leur forme de base : "manger" → "mangé"

5️⃣ **Vectorisation** TF-IDF

6️⃣ **Export** du fichier final pour l’entraînement IA.

# **📌 1 : Mise en minuscule et suppression des accents**
Convertir tout le texte en minuscules et remplacer les lettres accentuées sans les supprimer.

In [ ]:
import unicodedata

def normalize_text(text):
    """Met en minuscule et enlève les accents du texte."""
    text = str(text).lower()  # Mise en minuscule
    text = unicodedata.normalize("NFD", text).encode("ascii", "ignore").decode("utf-8")  # Suppression des accents
    return text

# Appliquer le nettoyage sur les titres et résumés
df["Titre_clean"] = df["Titre"].apply(normalize_text)
df["Résumé_clean"] = df["Résumé"].apply(normalize_text)

# Vérifier le résultat
print("✅ Exemple avant/après la normalisation :\n")
print("Avant :", df["Titre"].iloc[0])
print("Après :", df["Titre_clean"].iloc[0])


✅ Exemple avant/après la normalisation :

Avant : Taupe & Mulot. Atteindre les sommets
Après : taupe & mulot. atteindre les sommets


# 📌 **2 : Suppression des symboles mais conservation des nombres**
✔ Supprimer la ponctuation et les symboles.
✔ Conserver les nombres (dates, références utiles).

In [ ]:
import re

def remove_punctuation_keep_numbers(text):
    """Supprime la ponctuation et les symboles spéciaux, mais garde les nombres."""
    text = re.sub(r"[^\w\s\d]", "", text)  # Supprime la ponctuation mais garde les nombres
    return text

# Appliquer le nettoyage sur les colonnes
df["Titre_clean"] = df["Titre_clean"].apply(remove_punctuation_keep_numbers)
df["Résumé_clean"] = df["Résumé_clean"].apply(remove_punctuation_keep_numbers)

# Vérifier le résultat
print("✅ Exemple avant/après suppression des symboles (mais conservation des nombres) :\n")
print("Avant :", df["Titre"].iloc[0])
print("Après :", df["Titre_clean"].iloc[0])


✅ Exemple avant/après suppression des symboles (mais conservation des nombres) :

Avant : Taupe & Mulot. Atteindre les sommets
Après : taupe  mulot atteindre les sommets


# 📌**3 : Découpage du texte en mots (Tokenisation)**
Transformer le texte en une liste de mots (tokens) pour l’analyse NLP.

✔ Facilite l’analyse de chaque mot indépendamment.

✔ Prépare les données pour des modèles d’IA (TF-IDF).

In [ ]:
!pip install spacy
!python -m spacy download fr_core_news_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 61.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
from tqdm import tqdm

# Charger le modèle français
nlp = spacy.load("fr_core_news_sm")

def tokenize_text(text):
    """Tokenise un texte avec spacy"""
    doc = nlp(text)
    return [token.text for token in doc]

# Appliquer la tokenisation avec une barre de progression
tqdm.pandas(desc="🔄 Tokenisation des résumés")
df["tokens"] = df["Résumé_clean"].progress_apply(tokenize_text)

# Vérifier les résultats
print("✅ Exemple avant/après la tokenisation :\n")
print("Avant :", df["Résumé_clean"].iloc[0])
print("Après :", df["tokens"].iloc[0])


🔄 Tokenisation des résumés: 100%|██████████| 71213/71213 [31:48<00:00, 37.32it/s]

✅ Exemple avant/après la tokenisation :

Avant :  aaaaahhhh   mulot  mulot  ca va petit imprudent  combien de fois taije dit de toujours regarder ou tu poses le pied comme je le fais moimeme   aie aie aie  cesse donc de plaisanter taupe je crois bien que je me suis casse la patte taupe et mulot saiment plus que tout malgre leurs differences ils se voient tous les jours sentraident et partagent au gre des saisons escalade dans les arbres circuit de course dans le potager et bien dautres aventures  taupe ny voit vraiment pas grandchose mais son imagination et sa creativite sont sans limites il est un peu ronchon mais assez philosophe la personne en qui il a le plus confiance cest son meilleur ami mulot  ils sont inseparables taupe raffole du chocolat et de la tarte aux lombrics mulot est joyeux et enthousiaste applique en tout parfois un peu inquiet aussi il a toujours le bienetre de taupe a coeur et deploie des tresors dimagination pour ne jamais froisser son meilleur ami il vit tout pr

# **📌 4 : Suppression des stopwords**
Enlever les mots très fréquents mais peu utiles (ex: "le", "de", "et") pour améliorer l’apprentissage de l’IA.

In [ ]:
from spacy.lang.fr.stop_words import STOP_WORDS
from tqdm import tqdm

def remove_stopwords(tokens):
    """Supprime les stopwords de la liste de tokens."""
    return [word for word in tokens if word.lower() not in STOP_WORDS]

# Appliquer la suppression des stopwords avec une barre de progression
tqdm.pandas(desc="🧹 Suppression des stopwords")
df["tokens"] = df["tokens"].progress_apply(remove_stopwords)

# Vérifier les résultats
print("✅ Exemple avant/après la suppression des stopwords :\n")
print("Avant :", df["tokens"].iloc[0])
print("Après :", df["tokens"].iloc[0])


🧹 Suppression des stopwords: 100%|██████████| 71213/71213 [00:01<00:00, 48424.32it/s]


✅ Exemple avant/après la suppression des stopwords :

Avant : [' ', 'aaaaahhhh', '  ', 'mulot', ' ', 'mulot', ' ', 'ca', 'petit', 'imprudent', ' ', 'fois', 'taije', 'regarder', 'poses', 'pied', 'moimeme', '  ', ' ', 'cesse', 'plaisanter', 'taupe', 'crois', 'bien', 'casse', 'patte', 'taupe', 'mulot', 'saiment', 'differences', 'voient', 'jours', 'sentraident', 'partagent', 'gre', 'saisons', 'escalade', 'arbres', 'circuit', 'course', 'potager', 'bien', 'dautres', 'aventures', ' ', 'taupe', 'ny', 'voit', 'vraiment', 'grandchose', 'imagination', 'creativite', 'limites', 'ronchon', 'philosophe', 'confiance', 'cest', 'meilleur', 'ami', 'mulot', ' ', 'inseparables', 'taupe', 'raffole', 'chocolat', 'tarte', 'lombrics', 'mulot', 'joyeux', 'enthousiaste', 'applique', 'inquiet', 'bienetre', 'taupe', 'coeur', 'deploie', 'tresors', 'dimagination', 'jamais', 'froisser', 'meilleur', 'ami', 'vit', 'jour', 'passe', 'quil', 'rende', 'visite', 'nouvelle', 'occupation', 'partager']
Après : [' ', 'aaaaahhhh

# **📌 5 : Lemmatisation**
Réduire les mots à leur forme de base pour éviter la répétition inutile de variantes (ex: "courir", "court", "courait" deviennent "courir").

In [ ]:
def lemmatize_tokens(tokens):
    """Applique la lemmatisation sur les tokens en utilisant spaCy."""
    doc = nlp(" ".join(tokens))  # Reconvertir la liste en phrase pour traiter avec spaCy
    return [token.lemma_ for token in doc]  # Retourner les lemmes

# Appliquer la lemmatisation avec une barre de progression
tqdm.pandas(desc="🔄 Lemmatisation des tokens")
df["tokens"] = df["tokens"].progress_apply(lemmatize_tokens)

# Vérifier les résultats
print("✅ Exemple avant/après la lemmatisation :\n")
print("Avant :", df["tokens"].iloc[0])
print("Après :", df["tokens"].iloc[0])


🔄 Lemmatisation des tokens: 100%|██████████| 71213/71213 [20:20<00:00, 58.34it/s]


✅ Exemple avant/après la lemmatisation :

Avant : ['  ', 'aaaaahhhh', '   ', 'mulot', '  ', 'mulot', '  ', 'ca', 'petit', 'impruder', '  ', 'fois', 'taije', 'regarder', 'pose', 'pied', 'moimeme', '     ', 'cesse', 'plaisanter', 'taupe', 'croire', 'bien', 'casse', 'patt', 'taupe', 'mulot', 'saiment', 'difference', 'voir', 'jour', 'sentraider', 'partager', 'gre', 'saison', 'escalad', 'arbre', 'circuit', 'cours', 'potager', 'bien', 'dautre', 'aventure', '  ', 'taupe', 'ny', 'voir', 'vraiment', 'grandchose', 'imagination', 'creativite', 'limite', 'ronchon', 'philosophe', 'confiance', 'cest', 'meilleur', 'ami', 'mulot', '  ', 'inseparable', 'taupe', 'raffol', 'chocolat', 'tarte', 'lombric', 'mulot', 'joyeux', 'enthousiaste', 'appliqu', 'inquiet', 'bienetre', 'taupe', 'coeur', 'deploi', 'tresor', 'dimagination', 'jamais', 'froisser', 'meilleur', 'ami', 'voir', 'jour', 'passer', 'quil', 'rend', 'visite', 'nouveau', 'occupation', 'partager']
Après : ['  ', 'aaaaahhhh', '   ', 'mulot', '  ', 'm

# **📌 6 : Vectorisation des textes**
 Transformer les tokens en vecteurs numériques utilisables pour entraîner un modèle d'IA.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm

# Convertir les tokens en texte
df["text_clean"] = df["tokens"].apply(lambda tokens: " ".join(tokens))

# Initialiser le vectoriseur TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # On limite à 5000 features

# Appliquer TF-IDF
tqdm.pandas(desc="🔢 Application du TF-IDF")
tfidf_matrix = tfidf_vectorizer.fit_transform(df["text_clean"].progress_apply(str))

# Vérifier la taille de la matrice
print("✅ Matrice TF-IDF créée avec succès !")
print(f"📏 Dimensions : {tfidf_matrix.shape}")


🔢 Application du TF-IDF: 100%|██████████| 71213/71213 [00:00<00:00, 662585.65it/s]


✅ Matrice TF-IDF créée avec succès !
📏 Dimensions : (71213, 5000)


**Pourquoi utiliser TF-IDF ?**

✔ Pondère les mots en fonction de leur importance dans le corpus.

✔ Utile pour les modèles classiques de NLP (SVM, Naïve Bayes, etc.).

✔ Réduit l’impact des mots trop fréquents

# **📌 7 : Sauvegarde des résultats**

✔ Enregistrer le dataset pré-traité en .csv.

In [ ]:

# Enregistrer le dataset avec les tokens
df.to_csv("livres_pretraite.csv", index=False)

print("✅ Dataset pré-traité sauvegardé en CSV avec succès !")

from google.colab import files


# Télécharger le dataset pré-traité
files.download("livres_pretraite.csv")



✅ Modèle Word2Vec sauvegardé avec succès !
✅ Dataset pré-traité sauvegardé en CSV avec succès !


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>